## Install pacakges

In [1]:
# install libraries
# !pip install hazm
# !pip install pandas
# !pip install gensim
# !pip install tqdm
# !pip3 install sklearn
# !pip3 install seaborn
# !pip3 install networkx

In [2]:
!pip3 install --upgrade scipy networkx

You should consider upgrading via the 'c:\users\mahdi\anaconda3\python.exe -m pip install --upgrade pip' command.


## Import libraries

In [3]:
# libraries
import codecs
import os
import json
import tqdm
import random
from collections import Counter
from nltk import FreqDist
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
from gensim.models.fasttext import FastText
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
import networkx as nx
from sklearn import preprocessing
from hazm import *

C:\Users\Mahdi\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


## Load documents

In [4]:
DIRNAME = '../new-dataset-with-category'
data = []
for i in range(1, 31):
    with open(f'{DIRNAME}/namnak-{i}.json', 'r', encoding="utf-8") as f:
        data.extend(json.loads(f.read()))

In [5]:
i = 10
doc = data[i]
doc_text = ' '.join(doc['paragraphs'])

## Instantiate hazm normalizer and lemmatizer

In [6]:
normalizer = Normalizer()
lemmatizer = Lemmatizer()

 ## Stopwords

In [7]:
# Persian Stopwords
# https://github.com/sobhe/hazm/blob/master/hazm/data/stopwords.dat
stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('stopwords.txt','r','utf-8').readlines()]

custom_stop_words = [normalizer.normalize(x.strip()) for x in codecs.open('custom_stopwords.txt','r','utf-8').readlines()]

total_stop_words = custom_stop_words + stopwords

# Preprocess documents

In [8]:
replaced_doc = doc_text.replace('-', ' ')
replaced_doc = replaced_doc.replace('\u200f', '‌')

## Sentence and word tokenizer

In [9]:
sentences = sent_tokenize(normalizer.normalize(replaced_doc))
words_list = [[_ for _ in word_tokenize(t)  if _ not in total_stop_words] for t in sentences]
words_list = [[lemmatizer.lemmatize(tok) for tok in ls] for ls in words_list]

# Create similarity matrix

In [10]:
sent_num = len(words_list)
words_set = [set(ls) for ls in words_list]
similarity_mat = np.zeros((sent_num, sent_num), dtype=float)
treshold = 4

for i in range(sent_num):
    for j in range(sent_num):
        intersect_len = len(words_set[i].intersection(words_set[j]))
        similarity_mat[i][j] = 0 if (intersect_len < treshold or i == j) else (intersect_len ** (1.2)) / len(words_set[i])

## Normalize similarity matrix

In [11]:
similarity_mat_normalized = preprocessing.normalize(similarity_mat, norm='l1')

## Create networkx graph

In [12]:
graph = nx.from_numpy_array(similarity_mat_normalized)

# Implement page_rank algorithm

In [13]:
page_rank = nx.pagerank(graph, alpha=0.9)

In [14]:
page_rank

{0: 0.03542074224771863,
 1: 0.0064517107907474305,
 2: 0.0064517107907474305,
 3: 0.0064517107907474305,
 4: 0.0064517107907474305,
 5: 0.0064517107907474305,
 6: 0.0064517107907474305,
 7: 0.0064517107907474305,
 8: 0.0064517107907474305,
 9: 0.02233430320326379,
 10: 0.0064517107907474305,
 11: 0.0064517107907474305,
 12: 0.0064517107907474305,
 13: 0.0064517107907474305,
 14: 0.0064517107907474305,
 15: 0.0064517107907474305,
 16: 0.0730357596142907,
 17: 0.0064517107907474305,
 18: 0.0064517107907474305,
 19: 0.05836670538243607,
 20: 0.0645157285833059,
 21: 0.0064517107907474305,
 22: 0.0064517107907474305,
 23: 0.0064517107907474305,
 24: 0.0064517107907474305,
 25: 0.0064517107907474305,
 26: 0.07682270149873943,
 27: 0.0064517107907474305,
 28: 0.0064517107907474305,
 29: 0.0064517107907474305,
 30: 0.0064517107907474305,
 31: 0.1385567720185225,
 32: 0.0064517107907474305,
 33: 0.05572322687892342,
 34: 0.0064517107907474305,
 35: 0.0064517107907474305,
 36: 0.05572322687892

## Print top n sentences of document

In [15]:
def get_top_n(values, n=5):
    np_page_rank = np.array(list(values))
    top_n = np_page_rank.argsort()[-n:][::-1]
    return top_n

In [16]:
print(f"{doc['title']}")
print(doc['link'])
print('---------------')
for i in get_top_n(page_rank.values()):
    print(sentences[i])
    print('---------------')

برای درمان افسردگی چه بخوریم و چه نخوریم؟
https://namnak.com/مواد-غذایی-درمان-افسردگی.p23484
---------------
غذاهای دیگر که با افسردگی مبارزه می‌کنند، عبارتند از: دانه‌های گیاه آفتابگردان و چیا آووکادو پرتقال ماست گوشت ماهی چرب لوبیا آب تره غلات کامل درمان افسردگی با مواد غذایی، غذاهایی که نباید بخورید: متاسفانه زمانی که از افسردگی رنج می‌برید، ممکن است به غذاهای ناسالم علاقه داشته باشید.
---------------
مواد مغذی به فعالیت سیستم عصبی شما کمک می‌کنند و از اضطراب و افسردگی جلوگیری می‌کنند.
---------------
غذاهایی که خلق و خوی شما را بهبود میبخشد و افسردگی را درمان می‌کنند در ادامه این بخش از سلامت نمناک به توصیه‌های خوراکی برای درمان افسردگی و رفع اضطراب می‌پردازیم؛ غذاهای حاوی مواد مغذی که شما را شاد می‌کنند، می‌توانند به رژیم غذایی دیگری که برای افسردگی نیاز دارید، اضافه شوند.
---------------
سبزیجات با برگ‌های سبز تیره: یک مواد مغذی فوق العاده هستند، که به جلوگیری از التهاب کمک می‌کند.
---------------
گردو: غنی از امگا ۳ و اسید چرب است.
---------------


# Implement hits algorithm

In [17]:
hubs, authorities = nx.hits(graph)

In [18]:
get_top_n(hubs.values(), n=10)

array([31, 33, 36, 16,  0, 26,  9, 37, 19,  3], dtype=int64)

In [19]:
get_top_n(authorities.values(), n=10)

array([31, 33, 36, 16,  0, 26,  9, 37, 19,  3], dtype=int64)

## Print top n sentences of document

### Hubs

In [20]:
print(f"{doc['title']}")
print(doc['link'])
print('---------------')
for i in get_top_n(hubs.values()):
    print(sentences[i])
    print('---------------')

برای درمان افسردگی چه بخوریم و چه نخوریم؟
https://namnak.com/مواد-غذایی-درمان-افسردگی.p23484
---------------
غذاهای دیگر که با افسردگی مبارزه می‌کنند، عبارتند از: دانه‌های گیاه آفتابگردان و چیا آووکادو پرتقال ماست گوشت ماهی چرب لوبیا آب تره غلات کامل درمان افسردگی با مواد غذایی، غذاهایی که نباید بخورید: متاسفانه زمانی که از افسردگی رنج می‌برید، ممکن است به غذاهای ناسالم علاقه داشته باشید.
---------------
مانند: سوسیس قند و شکر و مشتقات آن‌ها نوشابه و سایر نوشیدنی‌های گازدار غذاهای سرخ شده مقدار بیش از حد کافئین شیرینی‌های تجاری پنیرهای چرب غلات غنی شده علاوه بر اینها، استفاده از روش‌های دیگر برای مبارزه با افسردگی بسیار مهم است.
---------------
غذایی که برای مبارزه و درمان با افسردگی باید هر روز مصرف کنیم: ۱.
---------------
غذاهایی که خلق و خوی شما را بهبود میبخشد و افسردگی را درمان می‌کنند در ادامه این بخش از سلامت نمناک به توصیه‌های خوراکی برای درمان افسردگی و رفع اضطراب می‌پردازیم؛ غذاهای حاوی مواد مغذی که شما را شاد می‌کنند، می‌توانند به رژیم غذایی دیگری که برای افسردگی نیاز دارید

### Authorities

In [21]:
print(f"{doc['title']}")
print(doc['link'])
print('---------------')
for i in get_top_n(authorities.values()):
    print(sentences[i])
    print('---------------')

برای درمان افسردگی چه بخوریم و چه نخوریم؟
https://namnak.com/مواد-غذایی-درمان-افسردگی.p23484
---------------
غذاهای دیگر که با افسردگی مبارزه می‌کنند، عبارتند از: دانه‌های گیاه آفتابگردان و چیا آووکادو پرتقال ماست گوشت ماهی چرب لوبیا آب تره غلات کامل درمان افسردگی با مواد غذایی، غذاهایی که نباید بخورید: متاسفانه زمانی که از افسردگی رنج می‌برید، ممکن است به غذاهای ناسالم علاقه داشته باشید.
---------------
مانند: سوسیس قند و شکر و مشتقات آن‌ها نوشابه و سایر نوشیدنی‌های گازدار غذاهای سرخ شده مقدار بیش از حد کافئین شیرینی‌های تجاری پنیرهای چرب غلات غنی شده علاوه بر اینها، استفاده از روش‌های دیگر برای مبارزه با افسردگی بسیار مهم است.
---------------
غذایی که برای مبارزه و درمان با افسردگی باید هر روز مصرف کنیم: ۱.
---------------
غذاهایی که خلق و خوی شما را بهبود میبخشد و افسردگی را درمان می‌کنند در ادامه این بخش از سلامت نمناک به توصیه‌های خوراکی برای درمان افسردگی و رفع اضطراب می‌پردازیم؛ غذاهای حاوی مواد مغذی که شما را شاد می‌کنند، می‌توانند به رژیم غذایی دیگری که برای افسردگی نیاز دارید

## check similarity matrix symmetricity

In [22]:
np.allclose(similarity_mat_normalized, similarity_mat_normalized.T, atol=1e-2)

False

# Link analysis using tfidf vectorizer

In [23]:
tfidfvectorizer = TfidfVectorizer(analyzer='word', stop_words=None, ngram_range=(1,1))
sent_term_mat = tfidfvectorizer.fit_transform([' '.join(sent) for sent in words_list])
similarity_mat = sent_term_mat.dot(sent_term_mat.T)
for i in range(similarity_mat.shape[0]):
    similarity_mat[i,i] = 0 

C:\Users\Mahdi\Anaconda3\lib\site-packages\scipy\sparse\_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [24]:
similarity_mat_normalized = preprocessing.normalize(similarity_mat, norm='l1')

graph = nx.from_numpy_array(similarity_mat_normalized)

page_rank = nx.pagerank(graph, alpha=0.9)

In [25]:
print(f"{doc['title']}")
print(doc['link'])
print('---------------')
for i in get_top_n(page_rank.values()):
    print(sentences[i])
    print('---------------')

برای درمان افسردگی چه بخوریم و چه نخوریم؟
https://namnak.com/مواد-غذایی-درمان-افسردگی.p23484
---------------
درمان افسردگی با مواد غذایی و تغذیه مناسب افسردگی یک مشکل سلامتی عمومی است، که بر میزان زیادی از جمعیت جهان تأثیر گذاشته است، با حالت غم انگیزی عمیق مشخص میشود، می‌تواند احساسات ناامیدی، از دست دادن علاقه به زندگی را افزایش دهد.
---------------
غذاهایی که خلق و خوی شما را بهبود میبخشد و افسردگی را درمان می‌کنند در ادامه این بخش از سلامت نمناک به توصیه‌های خوراکی برای درمان افسردگی و رفع اضطراب می‌پردازیم؛ غذاهای حاوی مواد مغذی که شما را شاد می‌کنند، می‌توانند به رژیم غذایی دیگری که برای افسردگی نیاز دارید، اضافه شوند.
---------------
ماهی‌ها برای مقابله با افسردگی نیز توصیه نیز می‌شوند چراکه کمبود نوعی از اسیدهای چرب موجود در آنها موجب تشدید علائم افسردگی می‌شود.
---------------
غذاهای دیگر که با افسردگی مبارزه می‌کنند، عبارتند از: دانه‌های گیاه آفتابگردان و چیا آووکادو پرتقال ماست گوشت ماهی چرب لوبیا آب تره غلات کامل درمان افسردگی با مواد غذایی، غذاهایی که نباید بخورید: متاسفان

## Link analyser final class

In [26]:
class LinkAnalyser:
    
    def __init__(self, treshold=4):
        self.normalizer = Normalizer()
        self.lemmatizer = Lemmatizer()
        self.treshold = treshold
        self.tfidfvectorizer = TfidfVectorizer(analyzer='word', stop_words=None, ngram_range=(1,1))
    
        stopwords = [self.normalizer.normalize(x.strip()) for x in codecs.open('../stopwords/stopwords.txt','r','utf-8').readlines()]
        custom_stop_words = [self.normalizer.normalize(x.strip()) for x in codecs.open('../stopwords/custom_stopwords.txt','r','utf-8').readlines()]
        self.total_stop_words = custom_stop_words + stopwords
        
    def get_top_n(self, values, n=5, descending=True):
        np_page_rank = np.array(list(values))
        if descending:
            top_n = np_page_rank.argsort()[-n:][::-1]
        else:
            top_n = np_page_rank.argsort()[:n]
        return top_n
    
    def pprint(self, ranking, doc, sentences, n=5, descending=True):
        for i in self.get_top_n(ranking.values(), n=n, descending=descending):
            print(sentences[i])
            print('---------------')
    
    def main_sentence_extractor(self, doc, similarity_type='words', method='pr', n=5):
        # method can be pr or hits
        # similarity_type can be words or tfidf
        doc_text = ' '.join(doc['paragraphs'])
        replaced_doc = doc_text.replace('-', ' ')
        replaced_doc = replaced_doc.replace('\u200f', '‌')
        sentences = sent_tokenize(self.normalizer.normalize(replaced_doc))
        words_list = [[_ for _ in word_tokenize(t)  if _ not in self.total_stop_words] for t in sentences]
        words_list = [[self.lemmatizer.lemmatize(tok) for tok in ls] for ls in words_list]
        
        if similarity_type == 'words':
        
            sent_num = len(words_list)
            words_set = [set(ls) for ls in words_list]
            similarity_mat = np.zeros((sent_num, sent_num), dtype=float)
            for i in range(sent_num):
                for j in range(sent_num):
                    intersect_len = len(words_set[i].intersection(words_set[j]))
                    similarity_mat[i][j] = 0 if (intersect_len < self.treshold or i == j) else (intersect_len ** (1.2)) / len(words_set[i])
        else:
            sent_term_mat = self.tfidfvectorizer.fit_transform([' '.join(sent) for sent in words_list])
            similarity_mat = sent_term_mat.dot(sent_term_mat.T)
            for i in range(similarity_mat.shape[0]):
                similarity_mat[i,i] = 0 
    
        similarity_mat_normalized = preprocessing.normalize(similarity_mat, norm='l1')
        graph = nx.from_numpy_array(similarity_mat_normalized)
        print(f"{doc['title']}")
        print(doc['link'])
        print('---------------')
        
        if method == 'pr':
            ranking = nx.pagerank(graph, alpha=0.9)
            print('Page rank:')
            print(f'top {n}')
            self.pprint(ranking, doc, sentences, n=n)
            print(f'bottom {n}')
            self.pprint(ranking, doc, sentences, n=n, descending=False)
        else:
            hubs, authorities = nx.hits(graph)
            print('Authorities:')
            print(f'top {n}')
            self.pprint(authorities, doc, sentences, n=n)
            print(f'bottom {n}')
            self.pprint(authorities, doc, sentences, n=n, descending=False)
            print('Hubs:')
            print(f'top {n}')
            self.pprint(hubs, doc, sentences, n=n)
            print(f'bottom {n}')
            self.pprint(hubs, doc, sentences, n=n, descending=False)


## Instantiate LinkAnalyser

In [27]:
link_analyser = LinkAnalyser()

## Word similarity with page rank method

In [28]:
i = 160
link_analyser.main_sentence_extractor(doc=data[i], similarity_type='words', method='pr', n=5)

اعتیاد به این مواد مخدر ترک ندارد!
https://namnak.com/اعتیاد-به-این-مواد-مخدر-ترک-ندارد.p20238
---------------
Page rank:
top 5
اعتیاد مواد مخدر و قرص‌های روانگردانی که غیرقابل ترک هستند ترکیبات شیمیایی جدیدی که در اشکال مختلف در دست خیلی از جوانان و نوجوانان داد و ستد می‌شود تا چند سال دیگر جای هر ماده مخدر و محرکی را که تاکنون وجود داشته است، می‌گیرد.
---------------
کسرا نام این ترکیبات را MA و MD عنوان کرده و می‌گوید: MA به شکل مایع و گرد با عنوان شیمیایی مواد مخدر تحت عنوان سوئیت، بنزای، مگاسوئیت و بیگ بنگ را می‌سازد و این اسم‌ها بستگی به میزان و موادی دارد که با این ماده ترکیب می‌شود در حالی که ماده اصلی این مخدر همان MA است.
---------------
به راحتی و در دست فرد مصرف کننده این ماده به روی سیگار یا هر ترکیبی به اندازه خیلی کم هم اگر ریخته شود کافی است و آن جوان و نوجوان تا روزها و حتی ساعت‌ها در حالت نشئگی به سر خواهد برد.
---------------
محمد طاهرخانی ادامه می‌دهد: دو ماده شیمیایی با نام‌های تجاری MD و MA اساس ساخت این مواد هستند که چون کاربردهای مختلفی دارند به صورت خام ماده مخ

## Word similarity with page hits method

In [29]:
link_analyser.main_sentence_extractor(data[i], method='hits', n=3)

اعتیاد به این مواد مخدر ترک ندارد!
https://namnak.com/اعتیاد-به-این-مواد-مخدر-ترک-ندارد.p20238
---------------
Authorities:
top 3
اعتیاد مواد مخدر و قرص‌های روانگردانی که غیرقابل ترک هستند ترکیبات شیمیایی جدیدی که در اشکال مختلف در دست خیلی از جوانان و نوجوانان داد و ستد می‌شود تا چند سال دیگر جای هر ماده مخدر و محرکی را که تاکنون وجود داشته است، می‌گیرد.
---------------
به راحتی و در دست فرد مصرف کننده این ماده به روی سیگار یا هر ترکیبی به اندازه خیلی کم هم اگر ریخته شود کافی است و آن جوان و نوجوان تا روزها و حتی ساعت‌ها در حالت نشئگی به سر خواهد برد.
---------------
متاسفانه علاوه بر پسران جوان، دختران نوجوان زیادی هم از سر تفنن حتی کمتر از پنج بار این مواد را مصرف کرده و برای همیشه سر از کلینیک ترک اعتیاد درآورده‌اند.
---------------
bottom 3
صحنه عجیبی است.
---------------
شیوع این مواد بین نوجوانان و در سنین مدرسه در همه جای دنیا رایج است.
---------------
این مواد به تازگی وارد بازار نشده‌اند بلکه علم و آگاهی درباره نحوه و ترکیبات آن در جوانان ایجاد شده و به همین واسطه اقبال عمومی

## Tfidf similarity with page hits method

In [30]:
link_analyser.main_sentence_extractor(data[160], similarity_type='tfidf')

اعتیاد به این مواد مخدر ترک ندارد!
https://namnak.com/اعتیاد-به-این-مواد-مخدر-ترک-ندارد.p20238
---------------
Page rank:
top 5
اعتیاد مواد مخدر و قرص‌های روانگردانی که غیرقابل ترک هستند ترکیبات شیمیایی جدیدی که در اشکال مختلف در دست خیلی از جوانان و نوجوانان داد و ستد می‌شود تا چند سال دیگر جای هر ماده مخدر و محرکی را که تاکنون وجود داشته است، می‌گیرد.
---------------
کسرا نام این ترکیبات را MA و MD عنوان کرده و می‌گوید: MA به شکل مایع و گرد با عنوان شیمیایی مواد مخدر تحت عنوان سوئیت، بنزای، مگاسوئیت و بیگ بنگ را می‌سازد و این اسم‌ها بستگی به میزان و موادی دارد که با این ماده ترکیب می‌شود در حالی که ماده اصلی این مخدر همان MA است.
---------------
این را پسری که این مواد را یک سال است مصرف می‌کند می‌گوید و ادامه می‌دهد: یعنی شما دیگر حتی به فروشنده این جنس هم نیاز ندارید.
---------------
کسرا به این صحنه می‌خندد و با صدای بلند یکی از پسرها را خطاب قرار داده و می‌گوید: ما همگی بچه مدرسه‌ای هستیم.
---------------
طاهرخانی می‌گوید: ظرف یکی دو سال گذشته این ترکیبات بین نوجوانان به ویژه بسی